In [5]:
# Load all necesearry packages
library(fpp3) 
library(quantmod)
#library(tsibble)
library(zoo)
#library(dplyr)
#library(ggplot2)
library(forecast)
#library(pastecs)
#install.packages("moments")
install.packages('moments')
library(moments)
#install.packages("tseries")
#tseries are very bad, goes against some fpp3 functions
#library(tseries)
#install.packages("sandwich")
library(sandwich)
#install.packages("rugarch") 
library(rugarch)
#install.packages("gridExtra") 
install.packages('moments')

library(gridExtra)



The downloaded binary packages are in
	/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T//Rtmp5rV7sf/downloaded_packages

The downloaded binary packages are in
	/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T//Rtmp5rV7sf/downloaded_packages


## UBS 

### UBS Data Cleaning and Transformation

In [6]:
library(quantmod)   # For stock data retrieval
library(tsibble)    # For working with time series data
library(rugarch)    # For GARCH modeling
library(tibble)     # For creating and manipulating data frames

symbol_ubs <- "UBS"
start_date <- "2018-05-08"
end_date <- "2023-05-08" 
# Retrieve the stock data using quantmod
getSymbols(symbol_ubs, from = start_date, to = end_date, src="yahoo")
stock_data_ubs <- UBS[, "UBS.Adjusted"]
log_returns_ubs <- diff(log(stock_data_ubs))
log_returns_ubs <- na.omit(log_returns_ubs)  # Remove any missing values

[1] "UBS"

In [7]:
library(PerformanceAnalytics)

VaR(log_returns_ubs,p = 0.90 ,method = "historical")
VaR(log_returns_ubs,p = 0.95 ,method = "historical")
VaR(log_returns_ubs,p = 0.99 ,method = "historical")


,UBS.Adjusted
VaR,-0.02117569


,UBS.Adjusted
VaR,-0.02985615


,UBS.Adjusted
VaR,-0.05506206


In [11]:
confidence_levels <- c(0.90, 0.95,0.99)

### VaR Uncondtional Mean

### VaR GARCH (1,1) - Normal Errors w. only a constant in the conditional mean function

In [12]:
# Create the GARCH(1,1) model
garch_spec_ubs <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(0, 0), include.mean = TRUE))

# Fit the GARCH model to the log returns
garch_model_ubs <- ugarchfit(spec = garch_spec_ubs, data = log_returns_ubs)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ubs <- ugarchforecast(garch_model_ubs, n.ahead = forecast_horizon)
forecasted_volatility_ubs <- sigma(forecast_ubs)
forecast_tbl_ubs <- tibble(Conditional_Mean = forecast_ubs@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ubs[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_ubs <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ubs[i]
  VaR_ubs[i, ] <- forecast_ubs@forecast$seriesFor[i] + qnorm(p = c(0.9,0.95,0.99),mean =0,sd=1)*forecast_ubs@forecast$sigmaFor[i]

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_ubs <- position * VaR_ubs

# Add VaR columns to the forecast table
forecast_tbl_ubs <- cbind(forecast_tbl_ubs, VaR = VaR_ubs, VaR_based_on_position = VaR_based_on_position_ubs)

# Print the final forecast table

forecast_tbl_ubs <- forecast_tbl_ubs %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_ubs

Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.0005994778,0.02424541,0.03167122,0.04047962,0.05700273,316712.2,404796.2,570027.3
0.0005994778,0.02417418,0.03157993,0.04036246,0.05683702,315799.3,403624.6,568370.2
0.0005994778,0.02410487,0.03149112,0.04024847,0.05667580,314911.2,402484.7,566758.0
0.0005994778,0.02403745,0.03140471,0.04013756,0.05651895,314047.1,401375.6,565189.5
0.0005994778,0.02397186,0.03132065,0.04002968,0.05636636,313206.5,400296.8,563663.6


For a GARCH(1,1), with normal errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for UBS increases. For a long position of 10,000,000 euros, we observe a roughly 80% increase in the VaR as the confidence level increases from 0.9 to 0,99. Conversely, over the five day ahead forecast horizon, the VaR decreases for each associated confidence level, although this is a small decrease of roughly 1%.

### VaR AR(1)- GARCH (1,1) - Normal Errors

In [13]:
garch_spec_ubs <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(1, 0), include.mean = TRUE))

# Fit the GARCH model to the log returns
garch_model_ubs <- ugarchfit(spec = garch_spec_ubs, data = log_returns_ubs)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ubs <- ugarchforecast(garch_model_ubs, n.ahead = forecast_horizon)
forecasted_volatility_ubs <- sigma(forecast_ubs)
forecast_tbl_ubs <- tibble(Conditional_Mean = forecast_ubs@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ubs[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_ubs <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ubs[i]
  VaR_ubs[i, ] <- forecast_ubs@forecast$seriesFor[i] + qnorm(p = c(0.9,0.95,0.99),mean =0,sd=1)*forecast_ubs@forecast$sigmaFor[i]

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_ubs <- position * VaR_ubs

# Add VaR columns to the forecast table
forecast_tbl_ubs <- cbind(forecast_tbl_ubs, VaR = VaR_ubs, VaR_based_on_position = VaR_based_on_position_ubs)

# Print the final forecast table
forecast_tbl_ubs <- forecast_tbl_ubs %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_ubs


Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.0025508311,0.02443195,0.03386164,0.04273782,0.05938806,338616.4,427378.2,593880.6
0.0007043788,0.02436173,0.03192519,0.04077585,0.05737823,319251.9,407758.5,573782.3
0.0006116090,0.02429336,0.03174480,0.04057063,0.05712642,317448.0,405706.3,571264.2
0.0006069480,0.02422682,0.03165486,0.04045652,0.05696695,316548.6,404565.2,569669.5
0.0006067138,0.02416205,0.03157163,0.04034975,0.05681605,315716.3,403497.5,568160.5


For an AR(1) - GARCH(1,1), with normal errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for UBS increases. For a long position of 10,000,000 euros, we observe a roughly 75% increase in the VaR as the confidence level increases from 0.9 to 0,99. Conversely, over the five day ahead forecast horizon, the VaR decreases for each associated confidence level, although this is a small decrease of roughly 1%.
When compared to the VaR obtained for the previous GARCH(1,1), we observe a substantially higher VaR at each associated confidence level. For example, the next day VaR at a 0.99 confidence level is approximately 4% larger, or 24,000 euros (approx.). 


### VaR GARCH (1,1) - STD-T Errors w. only a constant in the conditional mean function

In [14]:
# Create the GARCH(1,1) model
garch_spec_ubs <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(0, 0), include.mean = TRUE), distribution.model = 'std')

# Fit the GARCH model to the log returns
garch_model_ubs <- ugarchfit(spec = garch_spec_ubs, data = log_returns_ubs)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ubs <- ugarchforecast(garch_model_ubs, n.ahead = forecast_horizon)
forecasted_volatility_ubs <- sigma(forecast_ubs)
forecast_tbl_tdist_ubs <- tibble(Conditional_Mean = forecast_ubs@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ubs[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_tdist_ubs <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ubs[i]
  VaR_tdist_ubs[i, ] <- forecast_ubs@forecast$seriesFor[i] + forecast_ubs@forecast$sigmaFor[i]* qdist(distribution='std', shape=garch_model_ubs@fit[["coef"]][["shape"]], p=c(0.90,0.95,0.99))

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_tdis_ubs <- position * VaR_tdist_ubs

# Add VaR columns to the forecast table
forecast_tbl_tdist_ubs <- cbind(forecast_tbl_tdist_ubs, VaR = VaR_tdist_ubs, VaR_based_on_position = VaR_based_on_position_tdis_ubs)

# Print the final forecast table
forecast_tbl_tdist_ubs <- forecast_tbl_tdist_ubs %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_tdist_ubs


Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.0006499862,0.02434466,0.02820184,0.03841037,0.06437974,282018.4,384103.7,643797.4
0.0006499862,0.02429423,0.02814477,0.03833214,0.06424772,281447.7,383321.4,642477.2
0.0006499862,0.02424507,0.02808913,0.03825589,0.06411903,280891.3,382558.9,641190.3
0.0006499862,0.02419715,0.02803490,0.03818157,0.06399360,280349.0,381815.7,639936.0
0.0006499862,0.02415045,0.02798205,0.03810914,0.06387135,279820.5,381091.4,638713.5


For an GARCH(1,1), with student t errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for UBS also increases. For a long position of 10,000,000 euros, the we observe a roughly 128% increase in the VaR as the confidence level increases from 0.9 to 0,99. Conversely, over the five day ahead forecast horizon, the VaR decreases for each associated confidence level, although this is a small decrease of roughly 1%.
When compared to the VaR obtained for the previous GARCH(1,1) with normal errors, we observe a significantly higher VaR at a confidence level of 0.99 (12%; approx 73,000 euros), although the VaR at a confidence level of 0.90 is lower when compared to the GARCH model with normal errors. 

### VaR AR(1) - GARCH (1,1) - STD-T Errors

In [15]:
# Create the GARCH(1,1) model
garch_spec_ubs <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(1, 0), include.mean = TRUE), distribution.model = 'std')

# Fit the GARCH model to the log returns
garch_model_ubs_ar <- ugarchfit(spec = garch_spec_ubs, data = log_returns_ubs)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ubs_ar <- ugarchforecast(garch_model_ubs_ar, n.ahead = forecast_horizon)
forecasted_volatility_ar_ubs <- sigma(forecast_ubs_ar)
forecast_tbl_ubs_ar <- tibble(Conditional_Mean = forecast_ubs_ar@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ar_ubs[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_ar <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ar_ubs[i]
  VaR_ar[i, ] <- forecast_ubs_ar@forecast$seriesFor[i] + forecast_ubs_ar@forecast$sigmaFor[i]* qdist(distribution='std', shape=garch_model_ubs_ar@fit[["coef"]][["shape"]], p=c(0.90,0.95,0.99))

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_ar <- position * VaR_ar

# Add VaR columns to the forecast table
forecast_tbl_ubs_ar <- cbind(forecast_tbl_ubs_ar, VaR = VaR_ar, VaR_based_on_position = VaR_based_on_position_ar)

# Print the final forecast table
forecast_tbl_ubs_ar <- forecast_tbl_ubs_ar %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_ubs_ar


Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.0006543906,0.02434648,0.02820750,0.03841690,0.06438966,282075.0,384169.0,643896.6
0.0006496488,0.02429620,0.02814585,0.03833416,0.06425327,281458.5,383341.6,642532.7
0.0006496482,0.02424718,0.02809038,0.03825814,0.06412496,280903.8,382581.4,641249.6
0.0006496482,0.02419941,0.02803632,0.03818404,0.06399990,280363.2,381840.4,639999.0
0.0006496482,0.02415285,0.02798363,0.03811183,0.06387802,279836.3,381118.3,638780.2


In [16]:
(338609.8 - 282075.9) #/ 282075.9

[1] 56533.9

For an AR(1) - GARCH(1,1), with student t errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for UBS also increases. For a long position of 10,000,000 euros, we observe a roughly 128% increase in the VaR as the confidence level increases from 0.9 to 0,99. Conversely, over the five day ahead forecast horizon, the VaR decreases for each associated confidence level, although this is a small decrease of roughly 1%.
When compared to the VaR obtained for the previous AR(1) - GARCH(1,1) with normal errors, we observe a significantly higher VaR at a confidence level of 0.99 (8%; approx 50,000 euros), although the VaR at a confidence level of 0.90 is lower when compared to the model with normal errors.  

### VaR AR(1) - GARCH-M (1,1) - STD-T Errors

In [20]:
# Create the GARCH(1,1) model
garch_spec_ubs <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(1, 0), include.mean = TRUE, archm = TRUE), distribution.model = 'std')

# Fit the GARCH model to the log returns
garch_model_ubs_gm_tdist <- ugarchfit(spec = garch_spec_ubs, data = log_returns_ubs)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ubs_gm_tdist <- ugarchforecast(garch_model_ubs_gm_tdist, n.ahead = forecast_horizon)
forecasted_volatility_ubs_gm_tdist <- sigma(forecast_ubs_gm_tdist)
forecast_tbl_gm_tdist <- tibble(Conditional_Mean = forecast_ubs_gm_tdist@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ubs_gm_tdist[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_tdist_gm <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ubs_gm_tdist[i]
  VaR_tdist_gm[i, ] <- forecast_ubs_gm_tdist@forecast$seriesFor[i] + forecast_ubs_gm_tdist@forecast$sigmaFor[i]* qdist(distribution='std', shape=garch_model_ubs_gm_tdist@fit[["coef"]][["shape"]], p=c(0.90,0.95,0.99))

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_tdis_gm <- position * VaR_tdist_gm

# Add VaR columns to the forecast table
forecast_tbl_gm_tdist <- cbind(forecast_tbl_gm_tdist, VaR = VaR_tdist_gm, VaR_based_on_position = VaR_based_on_position_tdis_gm)

# Print the final forecast table
forecast_tbl_gm_tdist <- forecast_tbl_gm_tdist %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_gm_tdist


Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.001629116,0.02436145,0.02925309,0.03946138,0.06535313,292530.9,394613.8,653531.3
0.001714897,0.02431110,0.02928177,0.03946896,0.06530720,292817.7,394689.6,653072.0
0.001708141,0.02426197,0.02921930,0.03938590,0.06517192,292193.0,393859.0,651719.2
0.001701768,0.02421403,0.02915857,0.03930508,0.06504016,291585.7,393050.8,650401.6
0.001695550,0.02416726,0.02909932,0.03922623,0.06491160,290993.2,392262.3,649116.0


In [21]:
(653519.3 - 649099.4)/ 653519.3

[1] 0.006763228

For an AR(1) - GARCH-M(1,1), with student t errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for UBS also increases. For a long position of 10,000,000 euros, we observe a roughly 127% increase in the VaR as the confidence level increases from 0.9 to 0,99. Furthermore, as opposed to the previous models, the VaR has an incremental increase over the five day forecast horizon.
When compared to the VaR obtained for the previous AR(1) - GARCH(1,1) with student t errors, we observe a significantly lower VaR at all associated confidence levels. Furthermore, we observe a outlying trend in the GARCH-M model with regards to the VaR over the forecast horizon. 

### VaR AR(1) - GARCH-M (1,1) - Normal Errors

In [22]:
# Create the GARCH(1,1) model
garch_spec_ubs <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(1, 0), include.mean = TRUE, archm = TRUE))

# Fit the GARCH model to the log returns
garch_model_m_nrm <- ugarchfit(spec = garch_spec_ubs, data = log_returns_ubs)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_gm_nm <- ugarchforecast(garch_model_m_nrm, n.ahead = forecast_horizon)
forecasted_volatility_gm_nm <- sigma(forecast_gm_nm)
forecast_tbl_gm_nm <- tibble(Conditional_Mean = forecast_gm_nm@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_gm_nm[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_gm_nm <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_gm_nm[i]
  VaR_gm_nm[i, ] <- forecast_gm_nm@forecast$seriesFor[i] + qnorm(p = c(0.9,0.95,0.99),mean =0,sd=1)*forecast_gm_nm@forecast$sigmaFor[i]

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_gm_nm <- position * VaR_gm_nm

# Add VaR columns to the forecast table
forecast_tbl_gm_nm <- cbind(forecast_tbl_gm_nm, VaR = VaR_gm_nm, VaR_based_on_position = VaR_based_on_position_gm_nm)

# Print the final forecast table
forecast_tbl_gm_nm <- forecast_tbl_gm_nm %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_gm_nm



Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.003751194,0.02439671,0.03501683,0.04388021,0.06050643,350168.3,438802.1,605064.3
0.002029705,0.02432518,0.03320368,0.04204106,0.05861853,332036.8,420410.6,586185.3
0.001937222,0.02425553,0.03302194,0.04183402,0.05836403,330219.4,418340.2,583640.3
0.001922381,0.02418772,0.03292019,0.04170764,0.05819144,329201.9,417076.4,581914.4
0.001911505,0.02412171,0.03282472,0.04158818,0.05802699,328247.2,415881.8,580269.9


In [23]:
(605065.6 - 350169.7) / 350169.7

[1] 0.7279211

For an AR(1) - GARCH-M(1,1), with normal errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for UBS also increases. For a long position of 10,000,000 euros, we observe a roughly 82% increase in the VaR as the confidence level increases from 0.9 to 0,99. Furthermore,the VaR has an incremental increase over the five day forecast horizon.
When compared to the VaR obtained for the previous AR(1) - GARCH - M(1,1) with student t errors, we observe a significantly lower VaR at a confidence level of 0.99, while a higher VaR is observed for confidence level of 0.90. Furthermore, we observe a similar outlying trend in the GARCH-M model with regards to the increasing VaR over the forecast horizon. 

## Ryanair (RYAAY)

### Data Cleaning

In [24]:
library(quantmod)   # For stock data retrieval
library(tsibble)    # For working with time series data
library(rugarch)    # For GARCH modeling
library(tibble)     # For creating and manipulating data frames

symbol_ryan <- "RYAAY"
start_date <- "2018-05-08"
end_date <- "2023-05-08" 
# Retrieve the stock data using quantmod
getSymbols(symbol_ryan, from = start_date, to = end_date)
stock_data_ryan<- RYAAY[, "RYAAY.Adjusted"]
log_returns_ryan <- diff(log(stock_data_ryan))
log_returns_ryan <- na.omit(log_returns_ryan)  # Remove any missing values
confidence_levels <- c(0.9, 0.95, 0.99)

[1] "RYAAY"

### Unconditional VaR

In [25]:
library(PerformanceAnalytics)

VaR(log_returns_ryan,p = 0.90 ,method = "historical")
VaR(log_returns_ryan,p = 0.95 ,method = "historical")
VaR(log_returns_ryan,p = 0.99 ,method = "historical")


,RYAAY.Adjusted
VaR,-0.02664093


,RYAAY.Adjusted
VaR,-0.03730439


,RYAAY.Adjusted
VaR,-0.06695162


### GARCH (1,1) With Constant Mean Normal Errors

In [26]:
# Create the GARCH(1,1) model
garch_spec_ryan <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(0, 0), include.mean = TRUE))

# Fit the GARCH model to the log returns
garch_model_ryan <- ugarchfit(spec = garch_spec_ryan, data = log_returns_ryan)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ryan <- ugarchforecast(garch_model_ryan, n.ahead = forecast_horizon)
forecasted_volatility_ryan <- sigma(forecast_ryan)
forecast_tbl_ryan <- tibble(Conditional_Mean = forecast_ryan@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ryan[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_ryan <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ryan[i]
  VaR_ryan[i, ] <- forecast_ryan@forecast$seriesFor[i] + qnorm(p = c(0.9,0.95,0.99),mean =0,sd=1)*forecast_ryan@forecast$sigmaFor[i]

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_ryan <- position * VaR_ryan

# Add VaR columns to the forecast table
forecast_tbl_ryan <- cbind(forecast_tbl_ryan, VaR = VaR_ryan, VaR_based_on_position = VaR_based_on_position_ryan)

# Print the final forecast table

forecast_tbl_ryan <- forecast_tbl_ryan %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_ryan

Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7.463506e-05,0.02231751,0.02867568,0.03678368,0.05199294,286756.8,367836.8,519929.4
7.463506e-05,0.02243089,0.02882097,0.03697016,0.05225668,288209.7,369701.6,522566.8
7.463506e-05,0.02254005,0.02896087,0.03714972,0.05251063,289608.7,371497.2,525106.3
7.463506e-05,0.02264518,0.02909560,0.03732264,0.05275521,290956.0,373226.4,527552.1
7.463506e-05,0.02274645,0.02922538,0.03748922,0.05299079,292253.8,374892.2,529907.9


For a GARCH(1,1), with normal errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for RYAAY increases. Furthermore, the VaR increases for each associated confidence level over the forecast horizon. For a long position of 10,000,000 euros, we observe a roughly 81% increase in the VaR as the confidence level increases from 0.9 to 0,99. .

### VaR AR(1) - GARCH (1,1) With Constant Mean Normal Errors

In [27]:
garch_spec_ryan <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(1, 0), include.mean = TRUE))

# Fit the GARCH model to the log returns
garch_model_ryan <- ugarchfit(spec = garch_spec_ryan, data = log_returns_ryan)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ryan <- ugarchforecast(garch_model_ryan, n.ahead = forecast_horizon)
forecasted_volatility_ryan <- sigma(forecast_ryan)
forecast_tbl_ryan <- tibble(Conditional_Mean = forecast_ryan@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ryan[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_ryan <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ryan[i]
  VaR_ryan[i, ] <- forecast_ryan@forecast$seriesFor[i] + qnorm(p = c(0.9,0.95,0.99),mean =0,sd=1)*forecast_ryan@forecast$sigmaFor[i]

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_ryan <- position * VaR_ryan

# Add VaR columns to the forecast table
forecast_tbl_ryan <- cbind(forecast_tbl_ryan, VaR = VaR_ryan, VaR_based_on_position = VaR_based_on_position_ryan)

# Print the final forecast table
forecast_tbl_ryan <- forecast_tbl_ryan %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_ryan


Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1.105485e-03,0.02229107,0.02967264,0.03777104,0.05296228,296726.4,377710.4,529622.8
1.197061e-04,0.02240444,0.02883215,0.03697173,0.05224023,288321.5,369717.3,522402.3
7.422246e-05,0.02251363,0.02892660,0.03710585,0.05244876,289266.0,371058.5,524487.6
7.212385e-05,0.02261881,0.02905930,0.03727676,0.05269135,290593.0,372767.6,526913.5
7.202702e-05,0.02272016,0.02918908,0.03744336,0.05292702,291890.8,374433.6,529270.2


For an AR(1) - GARCH(1,1), with normal errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for RYAAY increases. Furthermore, the VaR increases for each associated confidence level over the forecast horizon For a long position of 10,000,000 euros, we observe a roughly 80% increase in the VaR as the confidence level increases from 0.9 to 0,99. 
When compared to the VaR obtained for the previous GARCH(1,1), we observe a  lower VaR at each associated confidence level.

### VaR GARCH (1,1) - STD-T Errors w. only a constant in the conditional mean function

In [28]:
# Create the GARCH(1,1) model
garch_spec_ryan <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(0, 0), include.mean = TRUE), distribution.model = 'std')

# Fit the GARCH model to the log returns
garch_model_ryan <- ugarchfit(spec = garch_spec_ryan, data = log_returns_ryan)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ryan <- ugarchforecast(garch_model_ryan, n.ahead = forecast_horizon)
forecasted_volatility_ryan <- sigma(forecast_ryan)
forecast_tbl_tdist_ryan <- tibble(Conditional_Mean = forecast_ryan@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ryan[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_tdist_ryan <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ryan[i]
  VaR_tdist_ryan[i, ] <- forecast_ryan@forecast$seriesFor[i] + forecast_ryan@forecast$sigmaFor[i]* qdist(distribution='std', shape=garch_model_ryan@fit[["coef"]][["shape"]], p=c(0.90,0.95,0.99))

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_tdis_ryan <- position * VaR_tdist_ryan

# Add VaR columns to the forecast table
forecast_tbl_tdist_ryan <- cbind(forecast_tbl_tdist_ryan, VaR = VaR_tdist_ryan, VaR_based_on_position = VaR_based_on_position_tdis_ryan)

# Print the final forecast table
forecast_tbl_tdist_ryan <- forecast_tbl_tdist_ryan %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_tdist_ryan


Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.0002450986,0.02110505,0.02308769,0.03202282,0.05617843,230876.9,320228.2,561784.3
0.0002450986,0.02115648,0.02314335,0.03210025,0.05631472,231433.5,321002.5,563147.2
0.0002450986,0.02120725,0.02319830,0.03217670,0.05644929,231983.0,321767.0,564492.9
0.0002450986,0.02125739,0.02325256,0.03225219,0.05658216,232525.6,322521.9,565821.6
0.0002450986,0.02130689,0.02330614,0.03232673,0.05671335,233061.4,323267.3,567133.5


In [29]:
(561784.3 - 519929.4 )#/ 519929.4

[1] 41854.9

For an GARCH(1,1), with student t errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for RYAAY also increases. For a long position of 10,000,000 euros, the we observe a roughly 141% increase in the VaR as the confidence level increases from 0.9 to 0,99. Further, over the five day ahead forecast horizon, the VaR increase for each associated confidence level.
When compared to the VaR obtained for the previous GARCH(1,1) with normal errors, we observe a significantly higher VaR at a confidence level of 0.99 (10%; approx 48,000 euros), although the VaR at a confidence level of 0.90 is only marginally higher when compared to the GARCH model with normal errors. 

### VaR AR(1) GARCH (1,1) - STD-T Errors

In [30]:
# Create the GARCH(1,1) model
garch_spec_ryan <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(1, 0), include.mean = TRUE), distribution.model = 'std')

# Fit the GARCH model to the log returns
garch_model_ryan_ar <- ugarchfit(spec = garch_spec_ryan, data = log_returns_ryan)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ryan_ar <- ugarchforecast(garch_model_ryan_ar, n.ahead = forecast_horizon)
forecasted_volatility_ar_ryan <- sigma(forecast_ryan_ar)
forecast_tbl_ryan_ar <- tibble(Conditional_Mean = forecast_ryan_ar@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ar_ryan[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_ar_tdist <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ar_ryan[i]
  VaR_ar_tdist[i, ] <- forecast_ryan_ar@forecast$seriesFor[i] + forecast_ryan_ar@forecast$sigmaFor[i]* qdist(distribution='std', shape=garch_model_ryan_ar@fit[["coef"]][["shape"]], p=c(0.90,0.95,0.99))

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_ar_ryan <- position * VaR_ar_tdist

# Add VaR columns to the forecast table
forecast_tbl_ryan_ar <- cbind(forecast_tbl_ryan_ar, VaR = VaR_ar_tdist, VaR_based_on_position = VaR_based_on_position_ar_ryan)

# Print the final forecast table
forecast_tbl_ryan_ar <- forecast_tbl_ryan_ar %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_ryan_ar


Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.0007591970,0.02110801,0.02359260,0.03252938,0.05670528,235926.0,325293.8,567052.8
0.0002467598,0.02115921,0.02313556,0.03209402,0.05632857,231355.6,320940.2,563285.7
0.0002346650,0.02120979,0.02317817,0.03215805,0.05645052,231781.7,321580.5,564505.2
0.0002343796,0.02125974,0.02323192,0.03223294,0.05658262,232319.2,322329.4,565826.2
0.0002343728,0.02130907,0.02328528,0.03230718,0.05671337,232852.8,323071.8,567133.7


In [31]:
(296726.4 - 235926.0) / 296726.4

[1] 0.2049039

For an AR(1) - GARCH(1,1), with student t errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for RYAAY also increases. For a long position of 10,000,000 euros, we observe a roughly 141% increase in the VaR as the confidence level increases from 0.9 to 0,99. Further, over the five day ahead forecast horizon, the VaR increases for each associated confidence level.
When compared to the VaR obtained for the previous AR(1) - GARCH(1,1) with normal errors, we observe a significantly higher VaR at a confidence level of 0.99 (10%; approx 48,000 euros), although the VaR at a confidence level of 0.90 is substantially lower when compared to the model with normal errors.  

### VaR AR(1) GARCH-M (1,1) - STD-T Errors

In [32]:
# Create the GARCH(1,1) model
garch_spec_ryan <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(1, 0), include.mean = TRUE, archm = TRUE), distribution.model = 'std')

# Fit the GARCH model to the log returns
garch_model_ryan_gm_tdist <- ugarchfit(spec = garch_spec_ryan, data = log_returns_ryan)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_ryan_gm_tdist <- ugarchforecast(garch_model_ryan_gm_tdist, n.ahead = forecast_horizon)
forecasted_volatility_ryan_gm_tdist <- sigma(forecast_ryan_gm_tdist)
forecast_tbl_gm_tdist_ryan <- tibble(Conditional_Mean = forecast_ryan_gm_tdist@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_ryan_gm_tdist[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_tdist_gm_ryan <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  forecasted_vol <- forecasted_volatility_ryan_gm_tdist[i]
  VaR_tdist_gm_ryan[i, ] <- forecast_ryan_gm_tdist@forecast$seriesFor[i] + forecast_ryan_gm_tdist@forecast$sigmaFor[i]* qdist(distribution='std', shape=garch_model_ryan_gm_tdist@fit[["coef"]][["shape"]], p=c(0.90,0.95,0.99))

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_tdis_gm_ryan <- position * VaR_tdist_gm_ryan

# Add VaR columns to the forecast table
forecast_tbl_gm_tdist_ryan <- cbind(forecast_tbl_gm_tdist_ryan, VaR = VaR_tdist_gm_ryan, VaR_based_on_position = VaR_based_on_position_tdis_gm_ryan)

# Print the final forecast table
forecast_tbl_gm_tdist_ryan <- forecast_tbl_gm_tdist_ryan %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_gm_tdist_ryan



Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.0006820692,0.02109487,0.02350681,0.03243785,0.05659112,235068.1,324378.5,565911.2
0.0001707697,0.02114570,0.02305052,0.03200308,0.05621456,230505.2,320030.8,562145.6
0.0001616413,0.02119592,0.02309572,0.03206955,0.05633852,230957.2,320695.5,563385.2
0.0001643279,0.02124552,0.02315208,0.03214690,0.05647267,231520.8,321469.0,564726.7
0.0001672580,0.02129452,0.02320803,0.03222360,0.05660547,232080.3,322236.0,566054.7


In [33]:
(565911.2 - 235068.1 )/ 235068.1

[1] 1.407435

For an AR(1) - GARCH-M(1,1), with student t errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for RYAAY also increases. For a long position of 10,000,000 euros, we observe a roughly 142% increase in the VaR as the confidence level increases from 0.9 to 0,99. Furthermore, the VaR has an incremental increase over the five day forecast horizon.
When compared to the VaR obtained for the previous AR(1) - GARCH(1,1) with student t errors, we observe a lower VaR at all associated confidence levels.

### VaR AR(1) GARCH-M (1,1) - Normal Errors

In [34]:
# Create the GARCH(1,1) model
garch_spec_ryan <- ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)),
                         mean.model = list(armaOrder = c(1, 0), include.mean = TRUE, archm = TRUE))

# Fit the GARCH model to the log returns
garch_model_m_nrm_ryan <- ugarchfit(spec = garch_spec_ryan, data = log_returns_ryan)

# Step 5: Obtain the forecast for the next 5 trading days
forecast_horizon <- 5
forecast_gm_nm_ryan <- ugarchforecast(garch_model_m_nrm_ryan, n.ahead = forecast_horizon)
forecasted_volatility_gm_nm_ryan <- sigma(forecast_gm_nm_ryan)
forecast_tbl_gm_nm_ryan <- tibble(Conditional_Mean = forecast_gm_nm_ryan@forecast$seriesFor, Forecasted_Volatility = forecasted_volatility_gm_nm_ryan[1:forecast_horizon])
# Step 6: Obtain the VaR for the next 5 trading days at different confidence levels
VaR_gm_nm_ryan <- matrix(0, nrow = forecast_horizon, ncol = length(confidence_levels))

for (i in 1:forecast_horizon) {
  print(i)
  forecasted_vol <- forecasted_volatility_gm_nm_ryan[i]
  VaR_gm_nm_ryan[i, ] <- forecast_gm_nm_ryan@forecast$seriesFor[i] + qnorm(p = c(0.9,0.95,0.99),mean =0,sd=1)*forecast_gm_nm_ryan@forecast$sigmaFor[i]

}

# Step 7: Calculate the VaR based on a long position of 10 million Euros
position <- 10e6
VaR_based_on_position_gm_nm_ryan <- position * VaR_gm_nm_ryan

# Add VaR columns to the forecast table
forecast_tbl_gm_nm_ryan <- cbind(forecast_tbl_gm_nm_ryan, VaR = VaR_gm_nm_ryan, VaR_based_on_position = VaR_based_on_position_gm_nm_ryan)

# Print the final forecast table
forecast_tbl_gm_nm_ryan <- forecast_tbl_gm_nm_ryan %>%
  rename(VaR90 = VaR.1,VaR95 = VaR.2,VaR99 = VaR.3, VaR90Pos = VaR_based_on_position.1,VaR95Pos = VaR_based_on_position.2,VaR99Pos = VaR_based_on_position.3)
forecast_tbl_gm_nm_ryan



[1] 1
[1] 2
[1] 3
[1] 4
[1] 5


Conditional_Mean,Forecasted_Volatility,VaR90,VaR95,VaR99,VaR90Pos,VaR95Pos,VaR99Pos
"<dbl[,1]>",<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9.890552e-04,0.02229287,0.02955852,0.03765756,0.05285003,295585.2,376575.6,528500.3
2.108596e-05,0.02240763,0.02873762,0.03687835,0.05214902,287376.2,368783.5,521490.2
-6.826231e-06,0.02251808,0.02885126,0.03703213,0.05237807,288512.6,370321.3,523780.7
7.732490e-06,0.02262443,0.02900210,0.03722160,0.05264002,290021.0,372216.0,526400.2
2.365086e-05,0.02272683,0.02914925,0.03740596,0.05289416,291492.5,374059.6,528941.6


For an AR(1) - GARCH-M(1,1), with normal errors, we observe that as the confidence level increases from 0.90 to 0.99, the Value at Risk (VaR) for RYAAY also increases. For a long position of 10,000,000 euros, we observe a roughly 82% increase in the VaR as the confidence level increases from 0.9 to 0,99. Furthermore,the VaR has an incremental increase over the five day forecast horizon.
When compared to the VaR obtained for the previous AR(1) - GARCH - M(1,1) with student t errors, we observe a significantly lower VaR at a confidence level of 0.99, while a higher VaR is observed for confidence level of 0.90. 

In [35]:
(528500.3 - 295585.2) / 295585.2

[1] 0.7879796

## MuliVariate

In [64]:
install.packages("rmgarch")
library(rmgarch)
# specify i.i.d. model for the univariate time series
ugarch_spec <- ugarchspec(mean.model = list(armaOrder = c(0,0), include.mean = FALSE), 
                          variance.model = list(model = "sGARCH", garchOrder = c(1,1)))
ms<-replicate(ugarch_spec, n = 2)
# specify DCC model
dcc_spec <- dccspec(uspec = multispec(ms),
                    VAR = TRUE, lag = 2,
                    model = "DCC", dccOrder = c(1,1))


The downloaded binary packages are in
	/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T//Rtmp5rV7sf/downloaded_packages


In [65]:
logreturns <- cbind(log_returns_ubs, log_returns_ryan)

In [66]:
logreturns

            UBS.Adjusted RYAAY.Adjusted
2018-05-09  0.0149071551  -0.0073761809
2018-05-10  0.0024631128  -0.0086898822
2018-05-11  0.0049079508   0.0032339320
2018-05-14 -0.0042931567  -0.0013462152
2018-05-15 -0.0043116674  -0.0131985194
2018-05-16 -0.0074350014   0.0058975991
2018-05-17 -0.0006220608  -0.0050788971
2018-05-18 -0.0062422114   0.0174857541
2018-05-21  0.0037499403   0.0478026119
2018-05-22  0.0191541574   0.0054366341
       ...                             
2023-04-24  0.0132191888  -0.0009502614
2023-04-25 -0.0478178257  -0.0186582526
2023-04-26  0.0182006545  -0.0267205713
2023-04-27  0.0173831056   0.0305873968
2023-04-28 -0.0029585328   0.0244624583
2023-05-01 -0.0104244674   0.0151581703
2023-05-02 -0.0065082080   0.0075959920
2023-05-03 -0.0285296986  -0.0227541623
2023-05-04 -0.0203609754  -0.0243552385
2023-05-05  0.0393018544   0.0224705050

In [67]:
garchdcc_fit <- dccfit(dcc_spec, data = logreturns)

In [77]:
dccforecast <- dccforecast(garchdcc_fit, n.ahead = 1)

In [69]:
forecast_gm_nm_ryan@forecast

,2023-05-05
T+1,0.02229287
T+2,0.02240763
T+3,0.02251808
T+4,0.02262443
T+5,0.02272683
,2023-05-05
T+1,9.890552e-04
T+2,2.108596e-05
T+3,-6.826231e-06
T+4,7.732490e-06


In [118]:
dccforecast


*---------------------------------*
*       DCC GARCH Forecast        *
*---------------------------------*

Distribution         :  mvnorm
Model                :  DCC(1,1)
Horizon              :  1
Roll Steps           :  0
-----------------------------------

0-roll forecast: 
, , 1

       [,1]   [,2]
[1,] 1.0000 0.5059
[2,] 0.5059 1.0000




In [87]:
dccforecast@mforecast

1.0000000,0.4654485
0.4654485,1.0000000


In [86]:
H <- dccforecast@mforecast$H
mu <- dccforecast@mforecast$mu

In [101]:
H_11<-H[[1]][1]
H_22<-H[[1]][2]


In [121]:
VAR11<- abs(mu[1] + qnorm(p = c(0.95),mean =0,sd=1)*H_11)
VAR22<- abs(mu[2] + qnorm(p = c(0.95),mean =0,sd=1)*H_22)


In [119]:
VAR22

[1] -0.0007156998

In [120]:
sqrt(0.00071569983078896)

[1] 0.02675257

In [122]:
covariance[1] <- cov(log_returns_ubs, log_returns_ryan)
corr <- covariance / (sqrt(VAR11) * sqrt(VAR22))

In [126]:
(-sqrt(VAR11^2 + VAR22^2 + corr*VAR11*VAR22)) * 10000000

,RYAAY.Adjusted
UBS.Adjusted,-40299.64
